In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from skl2onnx.helpers.onnx_helper import load_onnx_model
from stitchnet.stitchonnx.utils import Net
from glob import glob
from pathlib import Path

netsFiles = sorted(glob('_models/fragments/net*'))
nets = []
for i,netsFile in enumerate(netsFiles):
    fragmentFiles = sorted(glob(str(Path(netsFile)/'fragment*.onnx')))
    onnxFragments = []
    for fragmentFile in fragmentFiles:
        onnxFragment = load_onnx_model(fragmentFile)
        onnxFragments.append(onnxFragment)
    net1 = Net(onnxFragments, i)
    nets.append(net1)

In [6]:
from stitchnet.stitchonnx.utils import load_cats_and_dogs_dset,convert_imagenet_to_cat_dog_label
from stitchnet.stitchonnx.utils import accuracy_score_model,accuracy_score_net,load_dl
from stitchnet.stitchonnx.utils import generate_networks, ScoreMapper
from stitchnet.stitchonnx.report import Report
from stitchnet.stitchonnx.utils import evalulate_stitchnet

from tqdm import tqdm
import torch
import numpy as np
import os
from collections import defaultdict
import hashlib
import random

random.seed(51)
np.random.seed(24)
torch.manual_seed(77)

STITCH_BATCH_SIZE = 32 # todo study the effect
EVAL_BATCH_SIZE = 64
MAX_DEPTH = 10
THRESOULD = 0.8
TOTAL_THRESOULD = 0.6

RESULT_NAME = "result_paper_32_t2"

dataset_train = load_cats_and_dogs_dset("train")
dataset_val = load_cats_and_dogs_dset("test")

dl_score = load_dl(dataset_train, STITCH_BATCH_SIZE)
data_score,t = next(iter(dl_score))
data_score = data_score.numpy()

In [7]:
# range(1)
k = 0
if os.path.exists(f'./_results/{RESULT_NAME}.txt'):
    with open(f'./_results/{RESULT_NAME}.txt', 'r') as f:
        k = len(f.read().split('\n'))

In [8]:
from tqdm import tqdm

scoreMapper = ScoreMapper(nets, data_score)
with Report(EVAL_BATCH_SIZE, f'./_results/{RESULT_NAME}.txt', 'w') as report:
    # for _ in tqdm(range(50)):
    generator = generate_networks(nets, scoreMapper, data_score, 
                          threshold=THRESOULD, totalThreshold=TOTAL_THRESOULD, 
                          maxDepth=MAX_DEPTH, sample=False)
    for i,(s,net) in enumerate(generator):
        netname = f"_results/{RESULT_NAME}/net{k:03}_{i:03}"
        report.evaluate(nets, net, netname, s, dataset_val)
        net.save(netname)
        k += 1

/opt/anaconda3/envs/stitchnet/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


current depth: 1


100%|██████████| 5/5 [00:18<00:00,  3.64s/it]


potential next fragments: 43
potential next fragments after thresholding of 0.8: 1 ['1.0']
totalscore 0.9999999403953552
epoch 0 loss 1.1021206590245772e-09
epoch 1 loss 2.515985115720738e-10
epoch 2 loss 1.4094161655475326e-10
epoch 3 loss 1.2066557385959242e-10
epoch 4 loss 1.1312103364983035e-10
epoch 5 loss 1.0917467265540176e-10
epoch 6 loss 1.0701414879986353e-10
epoch 7 loss 1.0490820405647417e-10
epoch 8 loss 1.0395160757177913e-10
epoch 9 loss 1.0331025229013881e-10
current depth: 2


100%|██████████| 5/5 [00:20<00:00,  4.06s/it]


...

  0%|          | 0/80 [00:00<?, ?it/s]/opt/anaconda3/envs/stitchnet/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(
100%|██████████| 80/80 [01:03<00:00,  1.26it/s]


len(self.fragments) 1
saving to _results/result_paper_32_t2/net137_137
totalscore 0.6707551647635182
epoch 0 loss 0.0
epoch 1 loss 0.0
epoch 2 loss 0.0
epoch 3 loss 0.0
epoch 4 loss 0.0
epoch 5 loss 0.0
epoch 6 loss 0.0
epoch 7 loss 0.0
epoch 8 loss 0.0
epoch 9 loss 0.0


  0%|          | 0/80 [00:00<?, ?it/s]/opt/anaconda3/envs/stitchnet/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(
100%|██████████| 80/80 [01:00<00:00,  1.33it/s]


len(self.fragments) 1
saving to _results/result_paper_32_t2/net138_138
totalscore 0.6156040243489974
epoch 0 loss 0.0
epoch 1 loss 0.0
epoch 2 loss 0.0
epoch 3 loss 0.0
epoch 4 loss 0.0
epoch 5 loss 0.0
epoch 6 loss 0.0
epoch 7 loss 0.0
epoch 8 loss 0.0
epoch 9 loss 0.0


  0%|          | 0/80 [00:00<?, ?it/s]/opt/anaconda3/envs/stitchnet/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(
 92%|█████████▎| 74/80 [00:57<00:04,  1.30it/s]
Traceback (most recent call last):
  File "/var/folders/t9/yqdtb7bs2zv99b4dfp4q014m0000gp/T/ipykernel_10000/954954804.py", line 11, in <cell line: 4>
    report.evaluate(nets, net, netname, s, dataset_val)
  File "/Users/surat/NSTDA/stitchnet/stitchnet/stitchnet/stitchonnx/report.py", line 14, in evaluate
    accuracy = accuracy_score_net(net, dataset, bs=self.bs)
  File "/Users/surat/NSTDA/stitchnet/stitchnet/stitchnet/stitchonnx/utils.py", line 1129, in accuracy_score_net
    y = evalulate_stitchnet(net, x)
  File "/Users/surat/NSTDA/stitchnet/stitchnet/stitchnet/stitchonnx/utils.py", line 1034, in evalulate_stitchnet
    outputs = ort_